# 교통카드 빅데이터 통합정보시스템에서 정보를 가져온다


### 가져올 값은 서울, 인천, 경기도의 수단 통행량이다

### 2018년 1월부터 2024년 2월까지의 데이터를 가져온다
### 가져올 항목은 시도, 시군구, 이용자유형, 연월, 교통수단, 시간대이다

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
import lxml
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
from tqdm.notebook import tqdm
import warnings
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib
from datetime import datetime
import re
warnings.simplefilter(action="ignore", category=FutureWarning)

/tmp/ipykernel_16762/200955567.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
options = webdriver.ChromeOptions()

url = "https://stcis.go.kr/wps/main.do"
driver = webdriver.Chrome(service=Service("../driver/chromedriver"))

driver.get(url)

In [56]:
tab_menu = driver.find_element(By.CLASS_NAME,"tab_menu")
tab_menu1 = tab_menu.find_elements(By.TAG_NAME,"li")[0]
put_mouse = ActionChains(driver).move_to_element(tab_menu1)
put_mouse.perform()
time.sleep(0.1)


In [57]:
go_search = driver.find_element(By.ID, "submenu1")
use_search = go_search.find_elements(By.CLASS_NAME,"sub2menu")[0]
purpose_move = use_search.find_elements(By.TAG_NAME,"a")[0]
purpose_move.click()
driver.implicitly_wait(20)

In [58]:
date_tab = driver.find_element(By.ID,"divRdoDate")
date_tab.find_elements(By.CLASS_NAME,"searchDateGubun")[1].click()

In [60]:
driver.find_element(By.ID,"searchFromMonth").click()
time.sleep(0.1)
year_picker = driver.find_element(By.CLASS_NAME,"ui-datepicker-year")
year_picker.click()
year_list = year_picker.find_elements(By.TAG_NAME,"option")

In [61]:
month_picker = driver.find_element(By.CLASS_NAME,"ui-datepicker-month")
month_picker.click()
month_list = month_picker.find_elements(By.TAG_NAME,"option")

In [63]:
search_zone = driver.find_element(By.ID,"searchZoneSd")
search_zone.click()
search_list = search_zone.find_elements(By.TAG_NAME,"option")
search_use_list = [1,4,9]       #순서대로 서울, 인천 ,경기도
search_list[1].click()


In [64]:
zone_box = driver.find_element(By.ID,"divSgg")
zone_box.find_element(By.CLASS_NAME,"check_area").click()

In [65]:
search_btn = driver.find_element(By.ID,"btnSearch")
search_btn.click()
driver.implicitly_wait(20)

In [54]:
driver.find_elements(By.CLASS_NAME,"btn_type03")[0].click()

In [70]:
search_use_list = [1,4,9]       #순서대로 서울, 인천 ,경기도

for i in range(len(year_list)):
    j=0
    while j<len(month_list):
        tab_menu = driver.find_element(By.CLASS_NAME,"tab_menu")
        tab_menu1 = tab_menu.find_elements(By.TAG_NAME,"li")[0]
        put_mouse = ActionChains(driver).move_to_element(tab_menu1)
        put_mouse.perform()
        time.sleep(0.1)

        go_search = driver.find_element(By.ID, "submenu1")
        use_search = go_search.find_elements(By.CLASS_NAME,"sub2menu")[0]
        purpose_move = use_search.find_elements(By.TAG_NAME,"a")[0]
        purpose_move.click()
        driver.implicitly_wait(20)

        date_tab = driver.find_element(By.ID,"divRdoDate")
        date_tab.find_elements(By.CLASS_NAME,"searchDateGubun")[1].click()

        driver.find_element(By.ID,"searchFromMonth").click()
        time.sleep(0.1)

        year_picker = driver.find_element(By.CLASS_NAME,"ui-datepicker-year")
        year_picker.click()
        year_list = year_picker.find_elements(By.TAG_NAME,"option")
        year_list[i].click()

        month_picker = driver.find_element(By.CLASS_NAME,"ui-datepicker-month")
        month_picker.click()
        month_list = month_picker.find_elements(By.TAG_NAME,"option")
        month_list[j].click()

        driver.find_element(By.CLASS_NAME,"ui-datepicker-close.ui-state-default.ui-priority-primary.ui-corner-all").click()

        search_zone = driver.find_element(By.ID,"searchZoneSd")
        search_zone.click()
        search_list = search_zone.find_elements(By.TAG_NAME,"option")
        search_list[1].click()

        zone_box = driver.find_element(By.ID,"divSgg")
        zone_box.find_element(By.CLASS_NAME,"check_area").click()

        search_btn = driver.find_element(By.ID,"btnSearch")
        search_btn.click()
        driver.implicitly_wait(20)

        driver.find_elements(By.CLASS_NAME,"btn_type03")[0].click()

        j+=1

## 서울 데이터 따로 보관하고 인천 시작

In [71]:
search_use_list = [1,4,9]       #순서대로 서울, 인천 ,경기도

for i in range(len(year_list)):
    j=0
    while j<len(month_list):
        tab_menu = driver.find_element(By.CLASS_NAME,"tab_menu")
        tab_menu1 = tab_menu.find_elements(By.TAG_NAME,"li")[0]
        put_mouse = ActionChains(driver).move_to_element(tab_menu1)
        put_mouse.perform()
        time.sleep(0.1)

        go_search = driver.find_element(By.ID, "submenu1")
        use_search = go_search.find_elements(By.CLASS_NAME,"sub2menu")[0]
        purpose_move = use_search.find_elements(By.TAG_NAME,"a")[0]
        purpose_move.click()
        driver.implicitly_wait(20)

        date_tab = driver.find_element(By.ID,"divRdoDate")
        date_tab.find_elements(By.CLASS_NAME,"searchDateGubun")[1].click()

        driver.find_element(By.ID,"searchFromMonth").click()
        time.sleep(0.1)

        year_picker = driver.find_element(By.CLASS_NAME,"ui-datepicker-year")
        year_picker.click()
        year_list = year_picker.find_elements(By.TAG_NAME,"option")
        year_list[i].click()

        month_picker = driver.find_element(By.CLASS_NAME,"ui-datepicker-month")
        month_picker.click()
        month_list = month_picker.find_elements(By.TAG_NAME,"option")
        month_list[j].click()

        driver.find_element(By.CLASS_NAME,"ui-datepicker-close.ui-state-default.ui-priority-primary.ui-corner-all").click()

        search_zone = driver.find_element(By.ID,"searchZoneSd")
        search_zone.click()
        search_list = search_zone.find_elements(By.TAG_NAME,"option")
        search_list[4].click()

        zone_box = driver.find_element(By.ID,"divSgg")
        zone_box.find_element(By.CLASS_NAME,"check_area").click()

        search_btn = driver.find_element(By.ID,"btnSearch")
        search_btn.click()
        driver.implicitly_wait(20)

        driver.find_elements(By.CLASS_NAME,"btn_type03")[0].click()

        j+=1

## 인천도 옮기고 경기도 시작

In [72]:
search_use_list = [1,4,9]       #순서대로 서울, 인천 ,경기도

for i in range(len(year_list)):
    j=0
    while j<len(month_list):
        tab_menu = driver.find_element(By.CLASS_NAME,"tab_menu")
        tab_menu1 = tab_menu.find_elements(By.TAG_NAME,"li")[0]
        put_mouse = ActionChains(driver).move_to_element(tab_menu1)
        put_mouse.perform()
        time.sleep(0.1)

        go_search = driver.find_element(By.ID, "submenu1")
        use_search = go_search.find_elements(By.CLASS_NAME,"sub2menu")[0]
        purpose_move = use_search.find_elements(By.TAG_NAME,"a")[0]
        purpose_move.click()
        driver.implicitly_wait(30)

        date_tab = driver.find_element(By.ID,"divRdoDate")
        date_tab.find_elements(By.CLASS_NAME,"searchDateGubun")[1].click()

        driver.find_element(By.ID,"searchFromMonth").click()
        time.sleep(0.1)

        year_picker = driver.find_element(By.CLASS_NAME,"ui-datepicker-year")
        year_picker.click()
        year_list = year_picker.find_elements(By.TAG_NAME,"option")
        year_list[i].click()

        month_picker = driver.find_element(By.CLASS_NAME,"ui-datepicker-month")
        month_picker.click()
        month_list = month_picker.find_elements(By.TAG_NAME,"option")
        month_list[j].click()

        driver.find_element(By.CLASS_NAME,"ui-datepicker-close.ui-state-default.ui-priority-primary.ui-corner-all").click()

        search_zone = driver.find_element(By.ID,"searchZoneSd")
        search_zone.click()
        search_list = search_zone.find_elements(By.TAG_NAME,"option")
        search_list[9].click()

        zone_box = driver.find_element(By.ID,"divSgg")
        zone_box.find_element(By.CLASS_NAME,"check_area").click()

        search_btn = driver.find_element(By.ID,"btnSearch")
        search_btn.click()
        driver.implicitly_wait(30)

        driver.find_elements(By.CLASS_NAME,"btn_type03")[0].click()

        j+=1

In [2]:
path="../data/seoulsu/"
su_col_names = ['시도코드','시도','시군구','연월','이용자유형','교통수단','합계 발생량','합계 도착량']    #수단통행량 컬럼
for time in range(4,25):
    su_col_names.append(str(time)+'시 발생량')
    su_col_names.append(str(time)+'시 도착량')
for time in range(1,4):
    su_col_names.append(str(time)+'시 발생량')
    su_col_names.append(str(time)+'시 도착량')

su_col_names

['시도코드',
 '시도',
 '시군구',
 '연월',
 '이용자유형',
 '교통수단',
 '합계 발생량',
 '합계 도착량',
 '4시 발생량',
 '4시 도착량',
 '5시 발생량',
 '5시 도착량',
 '6시 발생량',
 '6시 도착량',
 '7시 발생량',
 '7시 도착량',
 '8시 발생량',
 '8시 도착량',
 '9시 발생량',
 '9시 도착량',
 '10시 발생량',
 '10시 도착량',
 '11시 발생량',
 '11시 도착량',
 '12시 발생량',
 '12시 도착량',
 '13시 발생량',
 '13시 도착량',
 '14시 발생량',
 '14시 도착량',
 '15시 발생량',
 '15시 도착량',
 '16시 발생량',
 '16시 도착량',
 '17시 발생량',
 '17시 도착량',
 '18시 발생량',
 '18시 도착량',
 '19시 발생량',
 '19시 도착량',
 '20시 발생량',
 '20시 도착량',
 '21시 발생량',
 '21시 도착량',
 '22시 발생량',
 '22시 도착량',
 '23시 발생량',
 '23시 도착량',
 '24시 발생량',
 '24시 도착량',
 '1시 발생량',
 '1시 도착량',
 '2시 발생량',
 '2시 도착량',
 '3시 발생량',
 '3시 도착량']

In [3]:
from glob import glob
file_paths = glob('/home/aa/dev_ws/git_ws/data/seoulsu/*')
file_paths

['/home/aa/dev_ws/git_ws/data/seoulsu/이용량 지표(수단통행량)_20240321 (58).xlsx',
 '/home/aa/dev_ws/git_ws/data/seoulsu/이용량 지표(수단통행량)_20240321 (71).xlsx',
 '/home/aa/dev_ws/git_ws/data/seoulsu/이용량 지표(수단통행량)_20240321 (59).xlsx',
 '/home/aa/dev_ws/git_ws/data/seoulsu/이용량 지표(수단통행량)_20240321 (73).xlsx',
 '/home/aa/dev_ws/git_ws/data/seoulsu/이용량 지표(수단통행량)_20240321 (62).xlsx',
 '/home/aa/dev_ws/git_ws/data/seoulsu/이용량 지표(수단통행량)_20240321 (14).xlsx',
 '/home/aa/dev_ws/git_ws/data/seoulsu/이용량 지표(수단통행량)_20240321 (2).xlsx',
 '/home/aa/dev_ws/git_ws/data/seoulsu/이용량 지표(수단통행량)_20240321 (4).xlsx',
 '/home/aa/dev_ws/git_ws/data/seoulsu/이용량 지표(수단통행량)_20240321 (11).xlsx',
 '/home/aa/dev_ws/git_ws/data/seoulsu/이용량 지표(수단통행량)_20240321 (51).xlsx',
 '/home/aa/dev_ws/git_ws/data/seoulsu/이용량 지표(수단통행량)_20240321 (1).xlsx',
 '/home/aa/dev_ws/git_ws/data/seoulsu/이용량 지표(수단통행량)_20240321 (35).xlsx',
 '/home/aa/dev_ws/git_ws/data/seoulsu/이용량 지표(수단통행량)_20240321 (3).xlsx',
 '/home/aa/dev_ws/git_ws/data/seoulsu/이용량 지표(수단통행량)_202

## 정렬이 안돼있다.

In [4]:
file_paths.sort(key = lambda x : int(x.split('(')[-1].split(')')[0]))
file_paths

['/home/aa/dev_ws/git_ws/data/seoulsu/이용량 지표(수단통행량)_20240321 (0).xlsx',
 '/home/aa/dev_ws/git_ws/data/seoulsu/이용량 지표(수단통행량)_20240321 (1).xlsx',
 '/home/aa/dev_ws/git_ws/data/seoulsu/이용량 지표(수단통행량)_20240321 (2).xlsx',
 '/home/aa/dev_ws/git_ws/data/seoulsu/이용량 지표(수단통행량)_20240321 (3).xlsx',
 '/home/aa/dev_ws/git_ws/data/seoulsu/이용량 지표(수단통행량)_20240321 (4).xlsx',
 '/home/aa/dev_ws/git_ws/data/seoulsu/이용량 지표(수단통행량)_20240321 (5).xlsx',
 '/home/aa/dev_ws/git_ws/data/seoulsu/이용량 지표(수단통행량)_20240321 (6).xlsx',
 '/home/aa/dev_ws/git_ws/data/seoulsu/이용량 지표(수단통행량)_20240321 (7).xlsx',
 '/home/aa/dev_ws/git_ws/data/seoulsu/이용량 지표(수단통행량)_20240321 (8).xlsx',
 '/home/aa/dev_ws/git_ws/data/seoulsu/이용량 지표(수단통행량)_20240321 (9).xlsx',
 '/home/aa/dev_ws/git_ws/data/seoulsu/이용량 지표(수단통행량)_20240321 (10).xlsx',
 '/home/aa/dev_ws/git_ws/data/seoulsu/이용량 지표(수단통행량)_20240321 (11).xlsx',
 '/home/aa/dev_ws/git_ws/data/seoulsu/이용량 지표(수단통행량)_20240321 (12).xlsx',
 '/home/aa/dev_ws/git_ws/data/seoulsu/이용량 지표(수단통행량)_20240321 

In [6]:
import pandas as pd
import numpy as np

seoul_data = []

for each_file in file_paths:
    seoul_one_data = pd.read_excel(each_file,header=1,names=su_col_names)   #서울 데이터 불러오기
    seoul_one_data.drop(seoul_one_data.index[0], inplace=True)              #합계 인덱스 제거
    seoul_data.append(seoul_one_data)

seoul_data = pd.concat(seoul_data)
seoul_data

,시도코드,시도,시군구,연월,이용자유형,교통수단,합계 발생량,합계 도착량,4시 발생량,4시 도착량,...,23시 발생량,23시 도착량,24시 발생량,24시 도착량,1시 발생량,1시 도착량,2시 발생량,2시 도착량,3시 발생량,3시 도착량
1,11,서울특별시,강남구,201801,경로,도시철도,1629740,1575091,335,12,...,5846,6772,626,1680,2,77,0,1,0,0
2,0,0,0,0,국가유공자,도시철도,25859,26034,1,1,...,361,236,50,90,0,3,0,0,0,0
3,0,0,0,0,0,시내,15051,6740,116,10,...,214,89,132,25,37,6,11,9,9,1
4,0,0,0,0,어린이,공항버스,344,0,20,0,...,5,0,0,0,0,0,0,0,1,0
5,0,0,0,0,0,광역,1109,897,0,0,...,13,2,5,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
742,0,0,0,0,청소년,광역,1091,946,0,0,...,39,17,16,12,12,7,0,0,0,0
743,0,0,0,0,0,도시철도,61858,60739,6,1,...,738,2826,97,650,1,25,0,0,0,0
744,0,0,0,0,0,마을,4088,4164,0,0,...,75,93,0,0,0,0,0,0,0,0
745,0,0,0,0,0,시내,177407,174272,131,102,...,3413,4331,1003,1491,202,319,71,124,29,51


## 인천 가져오기

In [7]:
file_paths = glob('/home/aa/dev_ws/git_ws/data/incheonsu/*')
file_paths.sort(key = lambda x : int(x.split('(')[-1].split(')')[0]))

In [8]:
incheon_data = []

for each_file in file_paths:
    incheon_one_data = pd.read_excel(each_file,header=1,names=su_col_names)   #인천 데이터 불러오기
    incheon_one_data.drop(incheon_one_data.index[0], inplace=True)              #합계 인덱스 제거
    incheon_data.append(incheon_one_data)

incheon_data = pd.concat(incheon_data)
incheon_data.head()

,시도코드,시도,시군구,연월,이용자유형,교통수단,합계 발생량,합계 도착량,4시 발생량,4시 도착량,...,23시 발생량,23시 도착량,24시 발생량,24시 도착량,1시 발생량,1시 도착량,2시 발생량,2시 도착량,3시 발생량,3시 도착량
1,28,인천광역시,강화군,201801,국가유공자,시내,848,415,14,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,어린이,광역,28,42,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,마을,6,4,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,시내,1432,1380,0,0,...,0,1,0,0,0,0,0,0,0,0
5,0,0,0,0,0,좌석,31,10,0,0,...,0,0,0,0,0,0,0,0,0,0


## 경기도 가져오기

In [9]:
file_paths = glob('/home/aa/dev_ws/git_ws/data/gyunggisu/*')
file_paths.sort(key = lambda x : int(x.split('(')[-1].split(')')[0]))

In [10]:
gyunggi_data = []

for each_file in file_paths:
    gyunggi_one_data = pd.read_excel(each_file,header=1,names=su_col_names)   #인천 데이터 불러오기
    gyunggi_one_data.drop(gyunggi_one_data.index[0], inplace=True)              #합계 인덱스 제거
    gyunggi_data.append(gyunggi_one_data)

gyunggi_data = pd.concat(gyunggi_data)
gyunggi_data.head()

,시도코드,시도,시군구,연월,이용자유형,교통수단,합계 발생량,합계 도착량,4시 발생량,4시 도착량,...,23시 발생량,23시 도착량,24시 발생량,24시 도착량,1시 발생량,1시 도착량,2시 발생량,2시 도착량,3시 발생량,3시 도착량
1,41,경기도,가평군,201801,경로,도시철도,25027,23993,5,2,...,101,228,1,77,0,0,0,0,0,0
2,0,0,0,0,국가유공자,도시철도,378,367,0,0,...,1,12,0,0,0,0,0,0,0,0
3,0,0,0,0,0,시내,14,11,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,어린이,광역,196,215,0,0,...,0,1,0,0,0,1,0,0,0,0
5,0,0,0,0,0,도시철도,1925,1832,0,0,...,2,12,0,0,0,0,0,0,0,0


In [11]:
all_data = pd.concat([seoul_data,incheon_data,gyunggi_data])
all_data["시도"].unique()

array(['서울특별시', 0, '인천광역시', '경기도'], dtype=object)

## 시도코드, 시도, 시군구, 연월, 이용자유형에서 0 제거

In [12]:
change_list = ['시도코드','시도','시군구','연월','이용자유형','교통수단']
all_data[change_list] = all_data[change_list].replace(0,pd.NA)
all_data[change_list] = all_data[change_list].fillna(method='ffill')
all_data

/tmp/ipykernel_19107/370571128.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  all_data[change_list] = all_data[change_list].fillna(method='ffill')
/tmp/ipykernel_19107/370571128.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  all_data[change_list] = all_data[change_list].fillna(method='ffill')


,시도코드,시도,시군구,연월,이용자유형,교통수단,합계 발생량,합계 도착량,4시 발생량,4시 도착량,...,23시 발생량,23시 도착량,24시 발생량,24시 도착량,1시 발생량,1시 도착량,2시 발생량,2시 도착량,3시 발생량,3시 도착량
1,11,서울특별시,강남구,201801,경로,도시철도,1629740,1575091,335,12,...,5846,6772,626,1680,2,77,0,1,0,0
2,11,서울특별시,강남구,201801,국가유공자,도시철도,25859,26034,1,1,...,361,236,50,90,0,3,0,0,0,0
3,11,서울특별시,강남구,201801,국가유공자,시내,15051,6740,116,10,...,214,89,132,25,37,6,11,9,9,1
4,11,서울특별시,강남구,201801,어린이,공항버스,344,0,20,0,...,5,0,0,0,0,0,0,0,1,0
5,11,서울특별시,강남구,201801,어린이,광역,1109,897,0,0,...,13,2,5,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
867,41,경기도,화성시,202402,청소년,광역,40485,39267,10,0,...,507,3814,254,1207,28,148,3,15,0,0
868,41,경기도,화성시,202402,청소년,도시철도,19040,19512,18,0,...,102,934,2,237,0,0,0,0,0,0
869,41,경기도,화성시,202402,청소년,마을,152574,150192,0,0,...,899,1148,492,572,242,251,130,159,14,25
870,41,경기도,화성시,202402,청소년,시내,213373,209477,15,5,...,3924,6357,684,1246,64,121,0,7,0,0


In [13]:
all_data.reset_index(drop=True)

,시도코드,시도,시군구,연월,이용자유형,교통수단,합계 발생량,합계 도착량,4시 발생량,4시 도착량,...,23시 발생량,23시 도착량,24시 발생량,24시 도착량,1시 발생량,1시 도착량,2시 발생량,2시 도착량,3시 발생량,3시 도착량
0,11,서울특별시,강남구,201801,경로,도시철도,1629740,1575091,335,12,...,5846,6772,626,1680,2,77,0,1,0,0
1,11,서울특별시,강남구,201801,국가유공자,도시철도,25859,26034,1,1,...,361,236,50,90,0,3,0,0,0,0
2,11,서울특별시,강남구,201801,국가유공자,시내,15051,6740,116,10,...,214,89,132,25,37,6,11,9,9,1
3,11,서울특별시,강남구,201801,어린이,공항버스,344,0,20,0,...,5,0,0,0,0,0,0,0,1,0
4,11,서울특별시,강남구,201801,어린이,광역,1109,897,0,0,...,13,2,5,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99081,41,경기도,화성시,202402,청소년,광역,40485,39267,10,0,...,507,3814,254,1207,28,148,3,15,0,0
99082,41,경기도,화성시,202402,청소년,도시철도,19040,19512,18,0,...,102,934,2,237,0,0,0,0,0,0
99083,41,경기도,화성시,202402,청소년,마을,152574,150192,0,0,...,899,1148,492,572,242,251,130,159,14,25
99084,41,경기도,화성시,202402,청소년,시내,213373,209477,15,5,...,3924,6357,684,1246,64,121,0,7,0,0


## sql로 데이터베이스에 옮기기

In [14]:
import mysql.connector

conn = mysql.connector.connect(
    host = "database-1.c3ieyo8m4v9o.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "eda_team",
    password = "123456",
    database = "eda_project"
)

In [15]:
cursor = conn.cursor()

### sql에선 컬럼 이름에 띄어쓰기를 하면 안되므로 띄어쓰기가 안된 컬럼을 새로 만들어준다

In [16]:
su_col_names = ['시도코드','시도','시군구','연월','이용자유형','교통수단','합계발생량','합계도착량']    #수단통행량 컬럼
for time in range(4,25):
    su_col_names.append(str(time)+'시발생량')
    su_col_names.append(str(time)+'시도착량')
for time in range(1,4):
    su_col_names.append(str(time)+'시발생량')
    su_col_names.append(str(time)+'시도착량')

## 데이터베이스에 테이블 만들기

In [88]:
sql="create table method_traffic (시도코드 int, 시도 varchar(16), 시군구 varchar(16), 연월 int, 이용자유형 varchar(16), 교통수단 varchar(16), 합계발생량 int, 합계도착량 int"
for column in su_col_names[8:]:
    sql = sql + ', ' + column + ' int'
sql=sql + ');'
sql

'create table method_traffic (시도코드 int, 시도 varchar(16), 시군구 varchar(16), 연월 int, 이용자유형 varchar(16), 교통수단 varchar(16), 합계발생량 int, 합계도착량 int, 4시발생량 int, 4시도착량 int, 5시발생량 int, 5시도착량 int, 6시발생량 int, 6시도착량 int, 7시발생량 int, 7시도착량 int, 8시발생량 int, 8시도착량 int, 9시발생량 int, 9시도착량 int, 10시발생량 int, 10시도착량 int, 11시발생량 int, 11시도착량 int, 12시발생량 int, 12시도착량 int, 13시발생량 int, 13시도착량 int, 14시발생량 int, 14시도착량 int, 15시발생량 int, 15시도착량 int, 16시발생량 int, 16시도착량 int, 17시발생량 int, 17시도착량 int, 18시발생량 int, 18시도착량 int, 19시발생량 int, 19시도착량 int, 20시발생량 int, 20시도착량 int, 21시발생량 int, 21시도착량 int, 22시발생량 int, 22시도착량 int, 23시발생량 int, 23시도착량 int, 24시발생량 int, 24시도착량 int, 1시발생량 int, 1시도착량 int, 2시발생량 int, 2시도착량 int, 3시발생량 int, 3시도착량 int);'

In [85]:
cursor.execute(sql)

In [20]:
from tqdm import tqdm
sql = "INSERT INTO method_traffic VALUES"
for i, row in tqdm(all_data.iterrows(), total = all_data.shape[0]):
    cursor.execute(sql+str(tuple(row)))
conn.commit()

100%|██████████| 99086/99086 [09:39<00:00, 171.01it/s]


In [21]:
cursor.close()
conn.close()